In [2]:
using JuMP
using HiGHS

# Datele problemei
CapacitateCentrale = [550, 650, 400, 300]  # Capacitățile maxime de producție (MW)
Costuri = [4, 5, 2, 1.5]  # Costuri de producție per unitate (lei/MW)
CosturiIntretinere = [1, 0.8, 0.3, 0.2]  # Costuri de întreținere per unitate (lei/MW) (Mentenanta)
EmisiiCO2 = [0.6, 0.2, 0.01, 0.01]  # Emisii de CO2 per unitate de energie (kg/MW)

# Pragurile și taxele pentru emisiile de CO2 (kg)
PraguriCO2 = [100, 150, 200]  # kg
TaxeCO2 = [200, 500, 900]  # lei

# Numele centralelor
NumeCentrale = ["termica", "hidroelectrica", "eoliana", "solara"]

# Scenarii de cerere de energie (MW)
CerereEnergie = [1100, 1300]


# Buclă pentru diferite scenarii de cerere de energie
for cerere in CerereEnergie
    println("\nScenariu pentru cerere de energie: $cerere MW")
    
    # Crearea modelului
    IC = Model(HiGHS.Optimizer)

    # Definirea variabilelor
    NumarCentrale = length(CapacitateCentrale)
    @variable(IC, x[1:NumarCentrale] >= 0)  # Energie produsă de fiecare centrală
    @variable(IC, emisii_totale_CO2 >= 0)  # Emisii totale de CO2

    # Variabile binare pentru pragurile de emisii
    @variable(IC, b[1:length(PraguriCO2)], Bin)

    # Definim funcția obiectiv
    @objective(IC, Max,
        6 * cerere - sum((Costuri[i] * x[i] + CosturiIntretinere[i] * x[i]) for i in 1:NumarCentrale) - sum(TaxeCO2[j] * b[j] for j in 1:length(PraguriCO2))
    )

    # Constrângerile pentru cererea de energie
    @constraint(IC, sum(x[i] for i in 1:NumarCentrale) >= cerere)
    for i in 1:NumarCentrale
        @constraint(IC, x[i] <= CapacitateCentrale[i])
    end

    # Constrângerea pentru emisiile totale de CO2
    @constraint(IC, emisii_totale_CO2 == sum(EmisiiCO2[i] * x[i] for i in 1:NumarCentrale))

    # Constrângerile pentru pragurile de emisii de CO2
    for j in 1:length(PraguriCO2)
        if j == 1
            @constraint(IC, emisii_totale_CO2 <= PraguriCO2[j] + b[j] * 1000)
        else
            @constraint(IC, emisii_totale_CO2 <= PraguriCO2[j] + b[j] * 1000)
            @constraint(IC, emisii_totale_CO2 >= PraguriCO2[j-1] + 1 - (1 - b[j]) * 1000)
        end
    end


    
    # Optimizarea modelului
    optimize!(IC)
    println("Status final: $(termination_status(IC))")
    
    if termination_status(IC) == MOI.OPTIMAL
        energie_totala = sum(value(x[i]) for i in 1:NumarCentrale)
        cost_total = sum((Costuri[i] * value(x[i]) + CosturiIntretinere[i] * value(x[i])) for i in 1:NumarCentrale)
        emisii_totale = value(emisii_totale_CO2)

        taxe_emisii = sum(TaxeCO2[j] * value(b[j]) for j in 1:length(PraguriCO2))

        cost_total += taxe_emisii
        profit_total = 6 * cerere - cost_total

        for i in 1:NumarCentrale
            energie_produsa = value(x[i])
            cost_producere = Costuri[i] * energie_produsa + CosturiIntretinere[i] * energie_produsa
            println("Energie produsă de centrala ", NumeCentrale[i], ": ", round(Int, energie_produsa), " MW")
            println("Costul de producere pentru centrala ", NumeCentrale[i], ": ", round(Int, cost_producere), " lei")
        end

        println("Incasarile ",  6*cerere , " lei")
        println("Costul total de producție a energiei: ", round(Int, cost_total), " lei")
        println("Emisiile totale de CO2: ", round(emisii_totale, digits=2), " kg")
        println("Taxele totale pentru emisii de CO2: ", round(Int, taxe_emisii), " lei")
        println("Profitul total: ", round(Int, profit_total), " lei")
    else
        println("Nicio soluție disponibilă pentru cererea de $cerere MW")
    end
end


Scenariu pentru cerere de energie: 1100 MW
Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-02, 1e+03]
  Cost   [2e+00, 9e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+02, 1e+03]
Presolving model
7 rows, 8 cols, 19 nonzeros  0s
6 rows, 7 cols, 29 nonzeros  0s

Solving MIP model with:
   6 rows
   7 cols (3 binary, 0 integer, 0 implied int., 4 continuous)
   29 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   6600            -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   6600            2776             137.75%        0      0      0         0     0.0s

33.3% inactive integer columns, restarting
Model after restart h